**Task 1 | MNIST dataset classification**

## Setup

In [ ]:
import numpy as np
import keras
from keras import layers
import tensorflow as tf

## Prepare the data

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Build the model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

## Train the model

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 43s 97ms/step - accuracy: 0.7808 - loss: 0.7473 - val_accuracy: 0.9803 - val_loss: 0.0789
Epoch 2/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 77s 87ms/step - accuracy: 0.9652 - loss: 0.1159 - val_accuracy: 0.9840 - val_loss: 0.0561
Epoch 3/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 42s 89ms/step - accuracy: 0.9731 - loss: 0.0858 - val_accuracy: 0.9885 - val_loss: 0.0443
Epoch 4/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 40s 87ms/step - accuracy: 0.9773 - loss: 0.0712 - val_accuracy: 0.9885 - val_loss: 0.0426
Epoch 5/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - accuracy: 0.9799 - loss: 0.0631 - val_accuracy: 0.9900 - val_loss: 0.0359
Epoch 6/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - accuracy: 0.9824 - loss: 0.0550 - val_accuracy: 0.9900 - val_loss: 0.0350
Epoch 7/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 39s 84ms/step - accuracy: 0.9838 - loss: 0.0512 - val_accuracy: 0.9903 - val_loss: 0.0361
Epoch 8/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 39s 93ms/step - accuracy: 0.9843 - loss: 0.0500 - 

## Download and save the model in .tflite

In [ ]:
model.export("save_model/")
converter = tf.lite.TFLiteConverter.from_saved_model("save_model/")
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at 'save_model/'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135742771680976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135742771681744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135742771681360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135742771684624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135742771685200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135742771685392: TensorSpec(shape=(), dtype=tf.resource, name=None)


## Evaluate the trained model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02484935335814953
Test accuracy: 0.9918000102043152
